In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import time

%matplotlib inline

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [3]:
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [4]:
batch_size = 50
n_epochs = 20

In [5]:
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [6]:
X = tf.placeholder(tf.float32, [None, 784], name='X_placeholder')
Y = tf.placeholder(tf.float32, [None, 10], name='Y_placeholder')

In [7]:
X_image = tf.reshape(X, [-1, 28, 28, 1])

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [12]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

Y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [13]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_conv))
optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(Y_conv, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs):
        flag = True
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        train_accuracy = accuracy.eval(feed_dict={X: X_batch, Y: Y_batch, keep_prob: 1.0})
        print "Epoch {0}, accuracy: {1}".format(i, train_accuracy)
        
        for _ in range(n_batches):
            if flag == True: 
                flag = False
            else:
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, cross_entropy], feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.5})
            
    accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
    print 'Test accuracy {0}'.format(accuracy_val)

#     for i in range(20000):
#       batch = mnist.train.next_batch(50)
#       if i%100 == 0:
#         train_accuracy = accuracy.eval(feed_dict={X:batch[0], Y: batch[1], keep_prob: 1.0})
#         print("step %d, training accuracy %g"%(i, train_accuracy))
#       optimizer.run(feed_dict={X: batch[0], Y: batch[1], keep_prob: 0.5})

#     print("test accuracy %g"%accuracy.eval(feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0}))

Epoch 0, accuracy: 0.0399999991059
Epoch 1, accuracy: 0.980000019073
Epoch 2, accuracy: 0.959999978542
Epoch 3, accuracy: 0.980000019073
Epoch 4, accuracy: 1.0
Epoch 5, accuracy: 0.959999978542
Epoch 6, accuracy: 1.0
Epoch 7, accuracy: 0.959999978542
Epoch 8, accuracy: 1.0
Epoch 9, accuracy: 1.0
Epoch 10, accuracy: 1.0
Epoch 11, accuracy: 1.0
Epoch 12, accuracy: 1.0
Epoch 13, accuracy: 1.0
Epoch 14, accuracy: 1.0
Epoch 15, accuracy: 1.0
Epoch 16, accuracy: 1.0
Epoch 17, accuracy: 1.0
Epoch 18, accuracy: 1.0
Epoch 19, accuracy: 1.0
Test accuracy 0.992500007153
